# diesel::update

```rust
pub fn update<T: IntoUpdateTarget>(
    source: T
) -> UpdateStatement<T::Table, T::WhereClause>
```
Создает инструкцию UPDATE. Когда таблица передается для обновления, каждая строка в таблице будет обновлена. Вы можете сузить эту область, вызвав фильтр для таблицы перед ее передачей, что приведет к UPDATE your_table SET ... WHERE args_to_filter.

Передача типа, который реализует Identifiable, аналогична передаче some_table.find(some_struct.id()).

## Struct diesel::query_builder::UpdateStatement

**set.** Предоставляет предложение SET инструкции UPDATE.

**filter.** Добавляет заданный предикат в предложение WHERE создаваемого оператора.

Если предложение WHERE уже есть, предикат будет дополнен оператором AND. Нет никакой разницы в поведении между update(table.filter(x)) и update(table).filter(x).

**into_boxed.** Упаковывает предложение WHERE этого оператора обновления.

Это полезно в случаях, когда вы хотите условно изменить запрос, но хотите, чтобы тип остался прежним. Серверная часть должна быть указана как часть этого. Невозможно упаковать запрос и использовать его на нескольких бэкэндах.

Упакованный запрос повлечет за собой незначительное снижение производительности, поскольку компилятор больше не может встраивать построитель запросов. Для большинства приложений эта стоимость будет минимальной.


# diesel::replace_into

```rust
pub fn replace_into<T>(target: T) -> IncompleteInsertStatement<T, Replace>
```
Создает инструкцию REPLACE.

Если есть нарушения ограничения, база данных вместо этого попытается заменить нарушающую строку. Эта функция доступна только для MySQL и SQLite.

# diesel::insert_into

```rus
pub fn insert_into<T>(target: T) -> IncompleteInsertStatement<T, Insert>
```
Создает инструкцию INSERT для целевой таблицы.

Вы можете добавить данные, вызвав values() или default_values(), как показано в примерах.

Бэкенды, поддерживающие предложение RETURNING, такие как PostgreSQL, могут возвращать вставленные строки, вызывая .get_results вместо .execute.

```rust
let rows_inserted = diesel::insert_into(users)
    .values(&name.eq("Sean"))
    .execute(&connection);

assert_eq!(Ok(1), rows_inserted);

let inserted_names = diesel::insert_into(users)
    .values(&vec![
        name.eq("Diva Plavalaguna"),
        name.eq("Father Vito Cornelius"),
    ])
    .returning(name)
    .get_results(&connection);
assert_eq!(Ok(vec!["Diva Plavalaguna".to_string(), "Father Vito Cornelius".to_string()]), inserted_names);

```

## Struct diesel::query_builder::IncompleteInsertStatement

Структура, возвращаемая функцией insert_into. Предоставляет методы values и default_values. Методы:

**default_values.** Вставляет ЗНАЧЕНИЯ ПО УМОЛЧАНИЮ в целевую таблицу.
```rust
pub fn default_values(self) -> InsertStatement<T, DefaultValues, Op>
```

Example:
```rust
connection.execute("CREATE TABLE users (
    name VARCHAR(255) NOT NULL DEFAULT 'Sean',
    hair_color VARCHAR(255) NOT NULL DEFAULT 'Green'
)")?;

insert_into(users)
    .default_values()
    .execute(&connection)
    .unwrap();
let inserted_user = users.first(&connection)?;
let expected_data = (String::from("Sean"), String::from("Green"));

assert_eq!(expected_data, inserted_user);
```

**values.** Вставляет заданные значения в таблицу, переданную в insert_into.

См. документацию по insert_into для примеров использования.

Этот метод иногда может выдавать чрезвычайно непрозрачные сообщения об ошибках из-за ограничений языка Rust. Если вы получаете сообщение об ошибке «overflow evaluating requirement» в результате вызова этого метода, вам может понадобиться & перед аргументом этого метода.
```rust
pub fn values<U>(self, records: U) -> InsertStatement<T, U::Values, Op>
```
